In [6]:
data = b"A" * (10**9)  # 1GB message

# Using PyNacl

In [7]:
# stdlib
import hashlib
import time

# third party
from nacl.signing import SigningKey

# Generate a new random signing key
signing_key = SigningKey.generate()

# Example large message
large_message = data

# Hash the message with SHA-256 using hashlib
start = time.time()
hash_object = hashlib.sha256()
hash_object.update(large_message)
hashed_message = hash_object.digest()
hash_time = time.time() - start

# Sign the hashed message with PyNaCl
start = time.time()
signed_hash = signing_key.sign(hashed_message)
sign_time = time.time() - start

# Directly sign the large message with PyNaCl
start = time.time()
signed_message = signing_key.sign(large_message)
direct_sign_time = time.time() - start

print(f"Time to hash with hashlib: {hash_time:.2f} seconds")
print(f"Time to sign hashed message with PyNaCl: {sign_time:.2f} seconds")
print(f"Total time (hash + sign): {hash_time + sign_time:.2f} seconds")
print(
    f"Time to directly sign large message with PyNaCl: {direct_sign_time:.2f} seconds"
)

Time to hash with hashlib: 0.45 seconds
Time to sign hashed message with PyNaCl: 0.00 seconds
Total time (hash + sign): 0.45 seconds
Time to directly sign large message with PyNaCl: 22.56 seconds


# Using Cryptography library

In [19]:
# !pip install cryptography

In [8]:
# third party
%%time
# third party
from cryptography.hazmat.primitives.asymmetric.ed25519 import Ed25519PrivateKey

private_key = Ed25519PrivateKey.generate()
signature = private_key.sign(data)

CPU times: user 1.38 s, sys: 11 ms, total: 1.39 s
Wall time: 1.38 s


In [20]:
# third party
%%time
# third party
from cryptography.hazmat.primitives.asymmetric.ed448 import Ed448PrivateKey

private_key = Ed448PrivateKey.generate()
signature = private_key.sign(data)

CPU times: user 2.22 s, sys: 23.2 ms, total: 2.24 s
Wall time: 2.25 s


In [22]:
# third party
%%time
# third party
from cryptography.hazmat.primitives import hashes
from cryptography.hazmat.primitives.asymmetric import dsa

private_key = dsa.generate_private_key(
    key_size=1024,
)
signature = private_key.sign(data, hashes.SHA256())

CPU times: user 485 ms, sys: 4.75 ms, total: 490 ms
Wall time: 489 ms


# Hashing by PyNacl

In [9]:
# third party
import nacl.encoding
import nacl.hash

methods = ["sha256", "sha512", "blake2b"]

for hash_method in methods:
    HASHER = getattr(nacl.hash, hash_method)

    start = time.time()
    digest = HASHER(data, encoder=nacl.encoding.HexEncoder)
    end = time.time()
    print(f"Time taken for {hash_method}", end - start)

Time taken for sha256 16.29426908493042
Time taken for sha512 11.238587856292725
Time taken for blake2b 7.366748094558716


# Hashing by cryptography library

In [18]:
# third party
from cryptography.hazmat.primitives import hashes

methods = ["SHA256", "SHA512", "BLAKE2b"]

for hash_method in methods:
    if hash_method == "BLAKE2b":
        digest = hashes.Hash(getattr(hashes, hash_method)(64))
    else:
        digest = hashes.Hash(getattr(hashes, hash_method)())

    start = time.time()
    digest.update(data)
    digest.finalize()
    end = time.time()
    print(f"Time taken for {hash_method}", end - start)

Time taken for SHA256 0.43844008445739746
Time taken for SHA512 0.6953341960906982
Time taken for BLAKE2b 7.246281862258911


# Hashing by python hashlib

In [17]:
methods = ["sha256", "sha512", "blake2b"]

for hash_method in methods:
    if hash_method == "blake2b":
        m = getattr(hashlib, hash_method)(digest_size=64)
    else:
        m = getattr(hashlib, hash_method)()

    start = time.time()
    m.update(data)
    m.digest()
    end = time.time()
    print(f"Time taken for {hash_method}", end - start)

Time taken for sha256 0.4372677803039551
Time taken for sha512 0.6927249431610107
Time taken for blake2b 1.4543838500976562
